In [140]:
import geopandas
import requests
import matplotlib.pyplot as plt
import what3words
import json
import pandas as pd
from pprint import pprint
from shapely.geometry import Point, Polygon, MultiLineString, box, mapping
from shapely.ops import unary_union
from scipy.spatial.distance import pdist, squareform

In [16]:
fields = geopandas.read_file('StoneHouseGrowPeriodswFields.geojson')
fields = fields.cx[:, :42.3]
field_name = 'BSF 3'
field = fields[fields.FIELD == field_name]
ff = set(zip(fields.FARM, fields.FIELD))
minx, miny, maxx, maxy = field.total_bounds

In [17]:
payload = {
    'key': 'EF6CNLGZ', 
    'bounding-box': f"{miny},{minx},{maxy},{maxx}",
    'format': 'geojson'
}

url = "https://api.what3words.com/v3/grid-section"
r = requests.get(url, params=payload)
grid = r.json()

In [4]:
mlstring = MultiLineString(grid["features"][0]["geometry"]["coordinates"])
mb = mlstring.buffer(0.0000001)
p = Polygon.from_bounds(*field.total_bounds)
diff = p.difference(mb)

6935


In [19]:
def centroids(polys, field, erosion_factor=0.00003):
    
    field = field.buffer(-1 * erosion_factor)

    def contains_centroid(poly, small_poly):
        p = small_poly.centroid
        # size of proper grid square
        if small_poly.area > 8e-10:
            return max(poly.contains(p))
        else:
            return False

    xx = [a.centroid for a in diff if contains_centroid(field, a)]
    return xx

In [65]:
aa = centroids(diff, field)

In [88]:
geocoder = what3words.Geocoder("EF6CNLGZ")

def collect_info(pt, farm="Sirmon Farm", field="BSF 3"):
    res = geocoder.convert_to_3wa(what3words.Coordinates(pt.y, pt.x))
    
    s = res['square']['southwest']['lat']
    w = res['square']['southwest']['lng']
    n = res['square']['northeast']['lat']
    e = res['square']['northeast']['lng']
    
    geodata = {
        "id": res["words"], 
        "w": w, 
        "s": s, 
        "e": e, 
        "n": n, 
        "center_x": pt.x, 
        "center_y": pt.y,
    }
    
    return geodata

df = pd.DataFrame([collect_info(i) for i in aa])

In [90]:
df.to_pickle("bsf3.pkl")

In [133]:
df = pd.read_pickle("bsf3.pkl")
size_parcel = 100
geovec = list(zip(df.center_x, df.center_y))
number_parcels = int(len(geovec)/size_parcel)
kmeans = KMeans(n_clusters=number_parcels, random_state=0).fit(geovec)
df["parcel"] = kmeans.labels_

In [145]:
df["poly"] = df[["w", "s", "e", "n"]].apply(lambda x: box(*x), axis=1)

parcel_ids = list(set(df.parcel))

features = []
for i in parcel_ids:
    parcel = df[df.parcel==i]
    geo = mapping(unary_union(parcel.poly))
    geo["properties"] = {"what3words": list(parcel.id)}
    features.append(geo)

In [149]:
geojson_data = {
    "type": "FeatureCollection",
    "properties": {
        "farm": "Sirmon Farm",
        "field": field_name
    },
    "features": features
}

with open('bsf3.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=4)


In [1]:
xs = [point.x for point in aa]
ys = [point.y for point in aa]

plt.rcParams['figure.figsize'] = [30, 15]
outer_bounds = Polygon.from_bounds(*field.total_bounds)
field.plot(color="#FAFAFA")
plt.scatter(xs, ys, s=10, c=kmeans.labels_)
plt.plot(*outer_bounds.exterior.xy)

NameError: name 'aa' is not defined